<a href="https://colab.research.google.com/github/allanbrunobr/crossover/blob/main/Copy_of_Solving_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Business Problems with AI

## Objective
Develop a proof of concept application that intelligently processes email order requests and customer inquiries for a fashion store. The system should categorize emails into product inquiries or order requests and generate appropriate responses based on product catalog information and stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

## Instructions

- All requirements have to be implemented with the use of LLMs to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Because solving this assessment requires using advanced LLM capabilities, we provide you with a temporary OpenAI API key granting access to GPT-4o. You can use this key or your own, but please note that the provided key has a token quota, so use it wisely. We have carefully adjusted the limits to ensure they are sufficient for you to complete the task.
- Address the requirements in the order listed. Please review the requirements ahead to have a general implementation plan in place before you start.
- Your deliverables should include the code developed within this notebook, and a spreadsheet containing results across separate sheets. Comments detailing your thought process are appreciated.
- You are allowed to use additional libraries (langchain, etc.) in order to make solution more concise.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output:**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process order requests in the order they are received. For each request, verify product availability in stock. If the order can be fully fulfilled, create a new order line with the status **created**. If the order cannot be fully fulfilled, create a line with the status **out of stock**. After placing the order, update the stock to accurately reflect the current inventory levels.

    **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate and save response emails based on order processing results. Depending on the order status email should inform customer that their order was processed or could not be fulfilled. If the order was successfully processed, send an email to the customer informing them that their order has been processed, including details like the product name and quantity. If the order could not be fulfilled due to insufficient stock, send an email explaining the situation and specifying which items are out of stock. Optionally, offer options such as waiting for restock or choosing alternative products. Do your best to make an email look production ready!

    **Output**: Populate the **order-response** sheet with columns: email ID, response.

### 3. Handle product inquiry

Customers may ask general open questions.
- Respond to product inquiries using the information from the product catalog.
- Ensure that solution you provide scales well to handle the full catalog of over 100,000 products. This means that solutions which include raw product catalog in the prompt will be rated lower!

**Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria

- **Utilization of AI Tools**: Effectiveness in leveraging AI technologies provided (e.g., OpenAI API) to fulfill the task requirements. Employ your knowledge of various AI driven development techniques depending on the task.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational.
- **Code Quality and Clarity**: Code should be well-organized with clear logic and structure.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the spreadsheet. Double check before submitting!

We look forward to seeing your solution and how you approach solving real-world problems using AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [28]:
# Install the OpenAI Python package.
%pip install openai

In [29]:
# Code example of OpenAI communication

from openai import OpenAI

clientOpenAI = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000001LnBdMAK'
)

completion = clientOpenAI.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hi there! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [30]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '1nKbLJQzj5Q-Kp3epIXlFB8WH1KB8Y3HtZ3WlxlBM8M0'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')
# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))



,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


# Task 1. Classify emails

In [43]:
!pip install gdown
!pip install fuzzywuzzy[speedup]

import gdown

# URL do Google Drive para download
url = 'https://drive.google.com/uc?id=1hLHBZ05s2GvnEVDiLEqmZjbPVkMDhs4E'
output = 'credentials.json'

gdown.download(url, output, quiet=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.0 MB/s eta 0:00:00


Downloading...
From: https://drive.google.com/uc?id=1hLHBZ05s2GvnEVDiLEqmZjbPVkMDhs4E
To: /content/credentials.json
100%|██████████| 2.38k/2.38k [00:00<00:00, 3.96MB/s]


'credentials.json'

In [51]:
import urllib.parse
import os
from google.colab import files
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd


scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file('credentials.json', scopes=scope)
client = gspread.authorize(creds)

def write_data_frame(df, document_id, sheet_name):
    try:
        # Open the Google Sheet
        sheet = client.open_by_key(document_id).worksheet(sheet_name)

        # Clear existing content in the sheet
        sheet.clear()

        # Write DataFrame to the sheet
        sheet.update([df.columns.values.tolist()] + df.values.tolist())

        print(f"Data successfully written to sheet '{sheet_name}'")
    except Exception as e:
        print(f"Error writing to sheet '{sheet_name}': {str(e)}")

def get_openai_response(prompt):
    response = clientOpenAI.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def classify_email(email_subject, message):
  prompt = f"""
  Classify the following email as either a "product inquiry" or an "order request"
  Email subject: {email_subject}
  Email body: {message}

  Respond with only one of these two categories: "product inquiry" or "order request"
  """
  return get_openai_response(prompt)


#Classify all e-mails
email_classification = emails_df[['email_id']].copy()
email_classification['category'] = emails_df.apply(
    lambda row: classify_email(row['subject'], row['message']), axis=1
)

# Save the results to email-classification sheet
write_data_frame(email_classification, document_id, 'email-classification')


Data successfully written to sheet 'email-classification'


# Task 2. Process order requests

In [53]:
import json
import re
import pandas as pd
from fuzzywuzzy import fuzz

def extract_max_quantity(quantity_str):
    """Extract the maximum quantity from a string that might contain a range."""
    numbers = re.findall(r'\d+', quantity_str)
    if numbers:
        return max(map(int, numbers))
    return 0

def clean_json_response(response):
    cleaned = re.sub(r'^```json\s*|\s*```$', '', response.strip())
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        print("Failed to parse JSON from the response.")
        return None

def singular_form(word):
    """Convert a potentially plural word to its singular form."""
    return re.sub(r's$', '', word)

def process_order(email_id, email_body):
    prompt = f"""
    Extract the product identifier (ID or name) and quantity from the following email:
    {email_body}

    The product identifier can be an alphanumeric code (e.g., RSG8901, SWL2345) or a product name.
    Provide the quantity exactly as mentioned in the email.
    If the quantity is "all remaining", respond with "all remaining" as the quantity.
    If you can't find a specific product identifier, respond with "UNKNOWN" for the product_id.

    Respond in JSON format. If multiple products are ordered, provide a list of objects:
    [
        {{
            "product_id": "extracted product identifier (code, name, or UNKNOWN)",
            "quantity": "quantity as mentioned in the email (can be a number, range, or 'all remaining')"
        }},
        ...
    ]
    """

    raw_response = get_openai_response(prompt)
    print(f"Raw OpenAI Response for email {email_id}:", raw_response)

    extraction_result = clean_json_response(raw_response)

    if extraction_result is None:
        return [{'email_id': email_id, 'product_id': None, 'quantity': 0, 'status': 'error'}]

    if not isinstance(extraction_result, list):
        extraction_result = [extraction_result]

    order_results = []
    for item in extraction_result:
        product_identifier = item.get('product_id', 'UNKNOWN')
        quantity_str = str(item.get('quantity', '0'))

        if product_identifier == 'UNKNOWN':
            order_results.append({'email_id': email_id, 'product_id': product_identifier, 'quantity': quantity_str, 'status': 'product not found'})
            continue

        # Try to match by product ID
        product = products_df[products_df['product_id'].str.upper() == product_identifier.upper().replace(' ', '')]

        # If not found, try to match by product name using fuzzy matching
        if product.empty:
            product_identifier_clean = re.sub(r'[^\w\s]', '', product_identifier.upper())
            products_df['name_clean'] = products_df['name'].apply(lambda x: re.sub(r'[^\w\s]', '', x.upper()))
            products_df['match_ratio'] = products_df['name_clean'].apply(lambda x: fuzz.partial_ratio(product_identifier_clean, x))
            best_match = products_df.loc[products_df['match_ratio'].idxmax()]

            if best_match['match_ratio'] > 80:  # Threshold for considering a match
                product = products_df[products_df['product_id'] == best_match['product_id']]

        if product.empty:
            order_results.append({'email_id': email_id, 'product_id': product_identifier, 'quantity': quantity_str, 'status': 'product not found'})
            continue

        product = product.iloc[0]
        available_stock = product['stock']

        if quantity_str.lower() == 'all remaining':
            interpreted_quantity = available_stock
        else:
            interpreted_quantity = extract_max_quantity(quantity_str)

        if available_stock >= interpreted_quantity:
            status = "created"
            products_df.loc[products_df['product_id'] == product['product_id'], 'stock'] -= interpreted_quantity
        else:
            status = "out of stock"

        order_results.append({
            'email_id': email_id,
            'product_id': product['product_id'],
            'quantity': str(quantity_str),  # Preserve the original quantity string
            'status': status
        })

    return order_results

def generate_order_response(order):
    product = products_df[products_df['product_id'] == order['product_id']]
    product_name = product.iloc[0]['name'] if not product.empty else "Unknown Product"

    prompt = f"""
    Generate a professional email response for an order with the following details:
    Product: {product_name}
    Quantity: {order['quantity']}
    Status: {order['status']}

    If the status is "created", inform the customer that their order has been processed successfully.
    If the status is "out of stock", explain the situation and offer options like waiting for restock or choosing alternative products.
    If the status is "product not found", apologize and ask for clarification on the product they were trying to order.

    Make the email look production-ready and professional.
    """
    return get_openai_response(prompt)
# Process order requests
order_requests = emails_df[emails_df['email_id'].isin(
    email_classification[email_classification['category'] == 'order request']['email_id']
)]

order_results = []
for _, email in order_requests.iterrows():
    order_results.extend(process_order(email['email_id'], email['message']))

# Save order status
order_status_df = pd.DataFrame(order_results)
write_data_frame(order_status_df, document_id, 'order-status')

# Generate and save response emails
order_responses = []
for order in order_results:
    response = generate_order_response(order)
    order_responses.append({'email_id': order['email_id'], 'response': response})

order_response_df = pd.DataFrame(order_responses)
write_data_frame(order_response_df, document_id, 'order-response')

Raw OpenAI Response for email E001: ```json
[
    {
        "product_id": "LTH0976 Leather Bifold Wallets",
        "quantity": "all remaining"
    }
]
```
Raw OpenAI Response for email E004: ```json
[
    {
        "product_id": "SFT1098 Infinity Scarves",
        "quantity": "three to four"
    }
]
```
Raw OpenAI Response for email E007: ```json
[
    {
        "product_id": "CLF2109 Cable Knit Beanies",
        "quantity": "5"
    },
    {
        "product_id": "FZZ1098 Fuzzy Slippers",
        "quantity": "2"
    }
]
```
Raw OpenAI Response for email E008: ```json
[
    {
        "product_id": "Versatile Scarves",
        "quantity": "one"
    }
]
```
Raw OpenAI Response for email E010: ```json
[
    {
        "product_id": "RSG8901 Retro Sunglasses",
        "quantity": "1 pair"
    }
]
```
Raw OpenAI Response for email E014: ```json
[
    {
        "product_id": "Sleek Wallet",
        "quantity": "1"
    }
]
```
Raw OpenAI Response for email E017: ```json
[
    {
        "produc

# Task 3. Handle product inquiry

In [55]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Combine relevant product information into a single string for each product
products_df['product_info'] = products_df.apply(lambda row: f"{row['product_id']} {row['name']} {row['category']} {row['description']}", axis=1)

# Create TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(products_df['product_info'])

def find_relevant_products(query, top_n=3):
    """Find the most relevant products based on the inquiry."""
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    related_product_indices = cosine_similarities.argsort()[:-top_n-1:-1]
    return products_df.iloc[related_product_indices]

def generate_inquiry_response(email_id, email_body):
    # Extract the inquiry from the email body
    prompt = f"""
    Extract the main product inquiry or question from the following email:
    {email_body}

    Provide a concise summary of the inquiry.
    """

    inquiry = get_openai_response(prompt)

    # Find relevant products
    relevant_products = find_relevant_products(inquiry)

    # Generate a response using the relevant product information
    product_info = relevant_products.apply(lambda row: f"Product ID: {row['product_id']}, Name: {row['name']}, Category: {row['category']}, Description: {row['description']}, Price: ${row['price']}, Stock: {row['stock']}", axis=1).tolist()

    response_prompt = f"""
    Generate a professional response to the following product inquiry:
    {inquiry}

    Use the following product information to inform your response:
    {' | '.join(product_info)}

    If the inquiry is about a specific product and it's in the provided information, focus on that product.
    If the inquiry is more general, provide an overview of the relevant products.
    If the inquiry asks about product availability or stock, include that information.
    Make the response informative and concise.
    """

    response = get_openai_response(response_prompt)

    return {'email_id': email_id, 'response': response}

# Process product inquiries
inquiry_requests = emails_df[emails_df['email_id'].isin(
    email_classification[email_classification['category'] == 'product inquiry']['email_id']
)]

inquiry_responses = []
for _, email in inquiry_requests.iterrows():
    inquiry_responses.append(generate_inquiry_response(email['email_id'], email['message']))

# Convert inquiry responses to a DataFrame
inquiry_response_df = pd.DataFrame(inquiry_responses)

# Save results to the "inquiry-response" sheet
write_data_frame(inquiry_response_df, document_id, 'inquiry-response')

Data successfully written to sheet 'inquiry-response'
